In [64]:
import pandas as pd
from numpy import array
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold,  GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

In [75]:
Xcsv = pd.read_csv('out_featers.csv', index_col='match_id')

In [76]:
n_samples = 1000#Xcsv.shape[0]#
X = Xcsv[0:n_samples]
X['first_blood_player1'].fillna(X['first_blood_player1'].median(), inplace=True)
X['first_blood_player2'].fillna(X['first_blood_player2'].median(), inplace=True)
X.fillna(300, inplace=True)
Y = X['radiant_win']
X.drop(['radiant_win',
        'duration',
        'tower_status_radiant', 
        'tower_status_dire', 
        'barracks_status_dire', 
        'barracks_status_radiant', 
        'start_time'],
       axis=1, inplace=True)


In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3,
    random_state = 241 )

In [78]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)
scorer = make_scorer(roc_auc_score)
params = {'n_estimators':[10*i for i in range(1, 4)],
         'learning_rate': [i/20 for i in range(4, 9)],
         'max_depth':[i for i in range(2, 5)]}
params = {'n_estimators':[30],
         'learning_rate': [0.25],
         'max_depth':[i for i in range(4, 8)]}
grid = GridSearchCV(GradientBoostingClassifier(random_state=241),
                    params, cv=cv, n_jobs=-1, scoring=scorer, verbose=True)
%time grid.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.5s finished


Wall time: 7.41 s


GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=241, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [30], 'learning_rate': [0.25], 'max_depth': [4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=True)

In [79]:
%time clf = GradientBoostingClassifier(**grid.best_params_, random_state=241, verbose=False ).fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
pred = clf.predict_proba(X_test)[:, 1]
roc_score = roc_auc_score(Y_test, pred)
for key in grid.best_params_:
    print(f'{key} : {grid.best_params_[key]}')
print(f'score = {score}')
print(f'ROC-score = {roc_score}')

Wall time: 622 ms
learning_rate : 0.25
max_depth : 7
n_estimators : 30
score = 0.6366666666666667
ROC-score = 0.672197689459833


clf_params = {'n_estimators':500, 'learning_rate': 0.25, 'max_depth':2}
%time clf = GradientBoostingClassifier(**clf_params, random_state=241, verbose=False ).fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
pred = clf.predict_proba(X_test)[:, 1]
roc_score = roc_auc_score(Y_test, pred)
for key in grid.best_params_:
    print(f'{key} : {grid.best_params_[key]}')
print(f'score = {score}')
print(f'ROC-score = {roc_score}')

clf = GradientBoostingClassifier(**clf_params, random_state=241, verbose=False ).fit(X, Y)

XT = pd.read_csv('test_featers.csv', index_col='match_id')
XT.drop(['start_time'], axis=1, inplace=True)
XT.fillna(3000, inplace=True) 

pred = clf.predict(XT)
with open('kaggle_out.csv', 'w') as kaggle:
    pass
with open('kaggle_out.csv', 'a') as kaggle:
    kaggle.writelines('match_id,radiant_win')
    for row in XT.iterrows():
        kaggle.writelines('\n')
        kaggle.writelines(','.join([str(row[0]), str(clf.predict_proba(array(row[1]).reshape(1, 101))[0][1]),]))

In [11]:
params = {'n_estimators':[10*i for i in range(1, 4)],
         'learning_rate': [i/20 for i in range(4, 9)],
         'max_depth':[i for i in range(2, 5)]}
params


{'learning_rate': [0.2, 0.25, 0.3, 0.35, 0.4],
 'max_depth': [2, 3, 4],
 'n_estimators': [10, 20, 30]}

In [12]:
Xh.shape

NameError: name 'Xh' is not defined